# Projet image Classification CNN in PyTorch 

Le but de ce projet est de mettre en application les réseaux de neurones et la classification d'image. L'algorithme doit être capable de classifier une image dans l'une des dix catégories crées.

In [2]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [3]:
transform = transforms.Compose([ #transforms.Compose permet de mettre plusieurs transformation au sein d'un même module
    transforms.ToTensor(),       
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transforms.Compose permet de mettre plusieurs transformation au sein d'un même module.

transforms.ToTensor() permet de passer, pour une image, des valeurs [0;255] à [0;1] qui est plus standard pour les réseaux de neurones.

transforms.Normalize permet de centrer les valeurs autours de 0 avec un écart-type de 0.5 soit [-1;1]

In [5]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

100%|██████████| 170498071/170498071 [01:02<00:00, 2745636.30it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


on télécharge deux datasets, un pour le train et un pour le test. Ils sont composés de 10 classes d'images (50 000 images de train et 10 000 de test) et on leur applique le transform.
Le DataLoader permet de créer des lots de 32 images pour les traiter en parallèle.

In [6]:
image, label = train_data[0]

In [7]:
image.size()

torch.Size([3, 32, 32])

Le 3 représente le RGB et 32x32 est la taille de l'image en pixels

In [8]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'frog', 'horse', 'ship', 'truck'] #c'est les 10 classes d'images

In [9]:
class NeuralNet(nn.Module): #on définit une classe NeuralNet qui hérite de nn.Module
    
    def __init__(self): #cst, on définit les couches de notre réseaux de neurones
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 12, 5) #3 est le nombre de feautures de notre image, 12 est le nombre de features map créées 
                                         #en sortie et 5 est la taille du kernel appliqué 5x5. Soit (12, 28, 28)
        self.pool = nn.MaxPool2d(2, 2) #extrait le max d'un carré de 2x2 pixels d'où (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5) #Soit (24, 10, 10) -> (24, 5, 5) le pool est réappliqué -> Flatten (24*5*5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120) #dense layer, fully connected layer
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) #il est obligatoire d'avoir 10 en outputs -> 10 classes d'images
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) #F.relu renvoit 0 si x<=0 et x si x>0 -> permet de casser la linéarité, améliore la capacité d'apprentissage
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

On a une nouvelle taille d'image: taille de l'image - taille du kernel -> 32-5=27

nouvelle taille / pas de déplacement du kernel -> 27/1 = 27

on ajoute 1 -> 27+1 = 28

Une couche entièrement connectée : Chaque neurone reçoit des entrées de tous les neurones de la couche précédente. Cela signifie que chaque neurone de la couche suivante utilise des informations de tous les neurones de la couche précédente pour calculer sa sortie. il y a un poids et un biais qui permet de faire une transformation linéaire.

In [11]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)#Stochastic Gradient Descent

La perte d'entropie croisée mesure la différence entre les probabilités prédites par le modèle et les véritables étiquettes de classe.

SGD permet de mettre à jour les paramètres de net pendant l'apprentissage. lr = learning rate, c'est la taille de mise à jour de poids à chaque itérations. momentum : aide à accélérer l'optimisation en tenant compte des mises à jour passées. Le momentum permet de "lisser" les mises à jour et d'aider à surmonter les petits minima locaux dans le paysage de perte.

In [13]:
for epoch in range(30):
    print(f'Training epoch {epoch}...')#epoch ou époque représente un passage entier dans le NN
    
    running_loss = 0.0 
    
    for i, data in enumerate(train_loader):
        inputs, labels = data #on sépare data en 2, l'image et son label
        
        optimizer.zero_grad() #PyTorch accumule les gradients par défaut, on les réinitialise pour pas qu'il y est d'influence
        
        outputs = net(inputs) #au début des ité, les outputs seront aléatoires puis vont s'améliorer à chaque ité
        
        loss = loss_function(outputs, labels)#on va chercher la perte entre la prédiction et le res attendu
        loss.backward() #Calcule les gradients pour tous les paramètres du modèle en fonction de la perte.
        optimizer.step() #Met à jour les poids du modèle en utilisant les gradients calculés et le taux d'apprentissage défini dans l'optimiseur.
        
        running_loss += loss.item() #.item convertit le tensor de la perte en float
        
    print(f'loss: {running_loss/len(train_loader):.4f}') #perte moyenne par lot avec 4 chiffres après la virgule

Training epoch 0...
loss: 2.2159
Training epoch 1...
loss: 1.7511
Training epoch 2...
loss: 1.5272
Training epoch 3...
loss: 1.4038
Training epoch 4...
loss: 1.3004
Training epoch 5...
loss: 1.2113
Training epoch 6...
loss: 1.1377
Training epoch 7...
loss: 1.0800
Training epoch 8...
loss: 1.0228
Training epoch 9...
loss: 0.9789
Training epoch 10...
loss: 0.9407
Training epoch 11...
loss: 0.8958
Training epoch 12...
loss: 0.8641
Training epoch 13...
loss: 0.8281
Training epoch 14...
loss: 0.7978
Training epoch 15...
loss: 0.7653
Training epoch 16...
loss: 0.7330
Training epoch 17...
loss: 0.7064
Training epoch 18...
loss: 0.6807
Training epoch 19...
loss: 0.6517
Training epoch 20...
loss: 0.6289
Training epoch 21...
loss: 0.6013
Training epoch 22...
loss: 0.5813
Training epoch 23...
loss: 0.5561
Training epoch 24...
loss: 0.5369
Training epoch 25...
loss: 0.5154
Training epoch 26...
loss: 0.4952
Training epoch 27...
loss: 0.4723
Training epoch 28...
loss: 0.4501
Training epoch 29...
los

On veut voir une convergence des epoch aux alentours de 0.3 ou 0.4

In [14]:
torch.save(net.state_dict(), 'trained_net.pth') #on sauvegarde l'état des poids du modèle

In [15]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [16]:
correct = 0 #compte le nombre de prédictions correctes faites par le modèle.
total = 0 #compte le nombre total d'exemples dans l'ensemble de test.

net.eval()

with torch.no_grad(): #on ne calcule pas les gradients, pas nécessaire pour l'évaluation, donc permet d'économiser du tps de calcul
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1) #trouve l'indice de la classe avec la plus haute probabilité dans les sorties du modèle.
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
accuracy = 100 * correct / total

print(f'Accuracy: {accuracy}%')

Accuracy: 69.15%


### On test le modèle sur 2 exemples

In [42]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),       
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = ['C:/Users/coren/Documents/PERSO/COURS_PERSO/bateau.jpg', 
               'C:/Users/coren/Documents/PERSO/COURS_PERSO/camion.jpg', 
               'C:/Users/coren/Documents/PERSO/COURS_PERSO/chevreuil.jpg', 
               'C:/Users/coren/Documents/PERSO/COURS_PERSO/chat.jpg', 
               'C:/Users/coren/Documents/PERSO/COURS_PERSO/cheval.jpg',
               'C:/Users/coren/Documents/PERSO/COURS_PERSO/grenouille.jpg',
               'C:/Users/coren/Documents/PERSO/COURS_PERSO/casserole.jpg',
               'C:/Users/coren/Documents/PERSO/COURS_PERSO/macron.jpg']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {class_names[predicted.item()-1]}')

Prediction: ship
Prediction: truck
Prediction: cat
Prediction: cat
Prediction: horse
Prediction: frog
Prediction: truck
Prediction: deer


On voit que l'algorithme a du mal a différencié le chevreuil du chat, mais sinon les résultats sont bons